In [4]:
import json
import re

import pandas as pd
import pymorphy2
from tqdm.notebook import tqdm as tn

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
russian_stopwords = nltk.corpus.stopwords.words('russian')
english_stopwords = nltk.corpus.stopwords.words('english')
word_tokenizer = nltk.WordPunctTokenizer()


data_path = r"C:\Users\User\Desktop\Study\2_sem\blek\data"

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
data_spbu = pd.read_csv(f"{data_path}/spbu_content.csv", header = None)
data_spbu.head(5)

,0,1,2
0,https://spbu.ru/,2023-04-14T00:56:50.6368499Z,Главная | Санкт-Петербургский государствен...
1,https://cabinet.spbu.ru/Account/LogOn/,2023-04-14T00:57:02.6226399Z,Вход ...
2,http://bb.spbu.ru/,2023-04-14T00:57:02.6225294Z,Blackboard Learn ...
3,https://events.spbu.ru/,2023-04-14T00:57:03.8827793Z,...
4,https://cabinet.spbu.ru/Account/Register/,2023-04-14T00:57:06.6414451Z,Регистраци...


In [4]:
data_msu = pd.read_csv(f"{data_path}/msu_content.csv", header = None)
data_msu.head(5)

,0,1,2
0,https://msu.ru/,2023-04-15T16:29:55.1662631Z,\t\tМосковский государственный университет име...
1,https://msu.ru/news/rss/,2023-04-15T16:30:13.1464229Z,"\tНовости МГУhttp://www.msu.ruSat, 15 Apr 2023..."
2,http://270.msu.ru/,2023-04-15T16:30:09.1446391Z,270 лет Московскому университету Указ Пре...
3,https://conf.msu.ru/rus/event/8147/,2023-04-15T16:30:09.1695835Z,...
4,http://nosh.msu.ru/,2023-04-15T16:30:11.2455821Z,Научно-образовательные школы Московского унив...


In [3]:
def has_cyrillic(text: str) -> bool: # проверяем, слово содержит русские буквы или нет 
    return bool(re.search('[а-яА-Я]', text))

def has_english(text: str) -> bool: # проверяем, слово содержит английские буквы или нет 
    return bool(re.search('[a-zA-z]', text))

In [11]:
data_spbu = data_spbu[~data_spbu[0].str.endswith('.pdf/')] # удаляем строки из датафрейма, которые пдфки
data_spbu = data_spbu.reset_index(drop=True) # сбрасываем индексы чтобы не возникало ошибок

In [12]:
data_spbu

,0,1,2
0,https://spbu.ru/,2023-04-14T00:56:50.6368499Z,Главная | Санкт-Петербургский государствен...
1,https://cabinet.spbu.ru/Account/LogOn/,2023-04-14T00:57:02.6226399Z,Вход ...
2,http://bb.spbu.ru/,2023-04-14T00:57:02.6225294Z,Blackboard Learn ...
3,https://events.spbu.ru/,2023-04-14T00:57:03.8827793Z,...
4,https://cabinet.spbu.ru/Account/Register/,2023-04-14T00:57:06.6414451Z,Регистраци...
...,...,...,...
30428,http://publishing.spbu.ru/catalog/tematicheski...,2023-04-15T14:12:41.5942241Z,\tКнига Арт-терапия в профилактической и леч...
30429,http://publishing.spbu.ru/catalog/periodichesk...,2023-04-15T14:13:48.1417110Z,\tЖурнал Новейшая история России - Издательств...
30430,http://publishing.spbu.ru/catalog/tematicheski...,2023-04-15T14:13:54.6711058Z,\tКнига Мышление и его расстройства при психич...
30431,http://publishing.spbu.ru/catalog/periodichesk...,2023-04-15T14:14:22.0093077Z,\tВестник СПбГУ. Международные отношения - Изд...


In [17]:
morph = pymorphy2.MorphAnalyzer()

def lemmatization(data_spbu):
    df = data_spbu.copy()
    list_errors = []
    for i in range(df.shape[0]):
        sub_string = data_spbu.iloc[i, 2]
        if type(sub_string) is str:
            word_list = nltk.word_tokenize(sub_string.lower()) # делаем маленькие буквы, разбиваем текст на токены (из 3-го столбика по каждой строке)

            word_list = [word for word in word_list if  (
                word not in russian_stopwords and
                word not in english_stopwords and
                not word.isnumeric() and 
                (has_cyrillic(word) or has_english(word))
                )
            ] # удаляем слова, которые в англ и русс стоп-словах или если они числовые (строчки)

            try:
                word_list = [morph.parse(j)[0].normal_form for j in word_list] # нормальная форма токена
                df.iloc[i, 2] = ' '.join(word_list) # заменяем 3 столбик на лемматизированный текст
            except:
                list_errors.append(i) # в лист ошибок добавляем номер строки, в которой ошибка 
    
    return df, list_errors

df, list_errors = lemmatization(data_spbu) # переменные из ретерна, которые возвращает функция 

In [33]:
def drop_errors(df, list_errors):
    return df.drop(index=list_errors) # удаляем все строки с ошибками

df = drop_errors(df=df, list_errors=list_errors)

df = df.reset_index(drop=True) # сбрасываем индексы чтобы не возникало ошибок

KeyError: '[426, 3306, 4132] not found in axis'

In [40]:
df.to_csv(f"{data_path}\df_spbu.csv") # сохраняем обработанный датафрейм в новый csv для последующего использования (токенизированный и почищенный)

In [6]:
df = pd.read_csv(f'{data_path}/df_spbu.csv', index_col=0,)

df.reset_index(inplace= True) #добавляем столбик индексов --> [0]

def create_dict(df):
    my_dict = {}

    for idx in range(df.shape[0]):  # цикл, который обрабатывает строки
        if isinstance(df.iloc[idx][3], str):
            tokens_list = list()
            tokens_list = df.iloc[idx][3].split() # если строка не встречается в листе ошибок (то что не получилось обработать на лемматизации)
            for token in tokens_list: # тут цикл, который обрабатывает токены (он смотрит, есть ли какая-то запись в словаре, соответствующая конкретному слову)
                if token in my_dict: # если ключ уже есть, то добавляем в список ссылку
                    my_dict[token].append(df.iloc[idx][0])
                else:  
                    my_dict[token] = [df.iloc[idx][0]] # пыталась тут добавить ключ и значение (то есть ссылку), если их нет в словаре 

    return my_dict

my_dict = create_dict(df)         
            # если запись в словаре есть - идем дальше, если записи в словаре нет - добавляем эту строку в лист для конкретного ключа
            # ну либо (если первый раз встречается) создаем лист, в который записывается значение строки для ключа

'def create_dict(df):\n    my_dict = {}\n\n    for idx in range(df.shape[0]):  # цикл, который обрабатывает строки\n        if isinstance(df.iloc[idx][2], str):\n            tokens_list = list()\n            tokens_list = df.iloc[idx][2].split() # если строка не встречается в листе ошибок (то что не получилось обработать на лемматизации)\n            for token in tokens_list: # тут цикл, который обрабатывает токены (он смотрит, есть ли какая-то запись в словаре, соответствующая конкретному слову)\n                if token in my_dict: # если ключ уже есть, то добавляем в список ссылку\n                    my_dict[token].append(df.iloc[idx][0])\n                else:  \n                    my_dict[token] = [df.iloc[idx][0]] # пыталась тут добавить ключ и значение (то есть ссылку), если их нет в словаре \n\n    return my_dict\n\nmy_dict = create_dict(df)   '

In [52]:
with open(f'{data_path}/result.json', 'w') as fp: # сохраняем словарь в json
    json.dump(my_dict, fp)

In [4]:
with open(f'{data_path}/result.json', 'r') as fp: #считываем словарь из json
    data = json.loads(fp.read())